<a href="https://colab.research.google.com/github/mbashish007/CommicSummarizer/blob/main/BartWithVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !curl -LO https://huggingface.co/datasets/VITA-MLLM/Comic-9K/resolve/main/images.tar.gz.aa

In [ ]:
# !tar -xvzf images.tar.gz.aa

In [ ]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!pip install -q pytesseract torchvision transformers datasets accelerate rouge-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
from transformers import BartTokenizer, BartForConditionalGeneration
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler
from torch.amp import autocast, GradScaler
from PIL import Image, ImageFile
import json
import os
import pytesseract
import re
import cv2
import string
import unicodedata
import numpy as np
from rouge_score import rouge_scorer
from tqdm import tqdm
from google.colab import drive
import matplotlib.pyplot as plt


In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
class Config:

    # Model architecture
    d_model = 512
    nhead = 4
    num_encoder_layers = 6
    num_decoder_layers = 6
    dim_feedforward = 1024
    max_text_len = 768
    max_len = 768
    max_images = 5
    image_size = 224
    resnet_spatial_size = 7  # For ResNet50 last conv layer

    # Training
    batch_size = 1
    learning_rate = 3e-5
    num_epochs = 5
    grad_accum_steps = 1
    save_path = "/content/drive/MyDrive/multimodal_summarization/"

    # Dataset
    json_file = "/content/drive/MyDrive/multimodal_summarization/comic_summary_chunks_small.json"
    json_file_small = "/content/drive/MyDrive/multimodal_summarization/comic_summary_chunks_small.json"
    test_json_file = "/content/drive/MyDrive/multimodal_summarization/comic_summary_test.json"

    image_dir = "/content/drive/MyDrive/multimodal_summarization/images"
    # image_dir = "/content/images"
    test_image_dir = "/content/drive/MyDrive/multimodal_summarization/test_images"
    pretrained_bart_path = "/content/drive/MyDrive/multimodal_summarization/xsum_step_4500_1745223373.pt"

    debug_samples = 5

config = Config()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def extract_comic_text(image_path):

    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text.strip()

In [ ]:
def clean_comic_text(text):

    if not text or not isinstance(text, str):
        return ""

    # Normalize unicode characters
    text = unicodedata.normalize('NFKD', text)

    # Remove non-printable characters
    text = ''.join(c for c in text if c.isprintable() or c in ['\n', '\t'])

    # Fix common OCR errors in comics
    replacements = {
        'l\'m': "I'm", 'l\'ll': "I'll", 'l\'ve': "I've", 'l\'d': "I'd",
        'lt': "It", 'ls': "Is", 'lf': "If", 'lnto': "Into", 'ln': "In",
        '0f': "Of", '0n': "On", '0r': "Or", '0ne': "One",
        'rnore': 'more', 'frorn': 'from', 'thern': 'them',
        '\\.\\.\\.\\.\\.': '...', '\\.\\.\\.\\.\\.\\.\\.': '...',
        '!!+': '!!', '\\?\\?+': '??',
        '\\[\\d+\\]': '', '\\(\\d+\\)': '',
    }

    # Apply all replacements
    for error, correction in replacements.items():
        text = re.sub(r'\b' + error + r'\b', correction, text, flags=re.IGNORECASE)

    # Fix spacing around punctuation
    text = re.sub(r'\s+([.,!?:;])', r'\1', text)

    # Fix capitalization
    # Capitalize first letter of sentences
    text = re.sub(r'(?<=[\.\?!]\s)([a-z])', lambda m: m.group(1).upper(), text)
    text = re.sub(r'^([a-z])', lambda m: m.group(1).upper(), text)

    # Fix common comic expressions
    text = re.sub(r'([Hh])a([Hh])a+', r'\1a\2a!', text)  # Fix laughter
    text = re.sub(r'([Aa])rgh+', r'\1rgh!', text)  # Fix screams

    # Remove repeated punctuation (except for ellipsis and common expressions)
    text = re.sub(r'([.,!?:;])\1{2,}', r'\1\1', text)
    text = re.sub(r'\.{2,5}', '...', text)  # Normalize ellipsis

    # Fix common OCR artifacts
    text = re.sub(r'[|]', 'I', text)  # Vertical bar to 'I'
    text = re.sub(r'[\u201C\u201D]', '"', text)  # Normalize quotes
    text = re.sub(r'[\u2018\u2019]', "'", text)  # Normalize apostrophes

    # Remove lines that are likely noise
    lines = text.split('\n')
    filtered_lines = []
    for line in lines:
        line = line.strip()

        if len(line) > 2 and any(c.isalpha() for c in line):
            alpha_ratio = sum(c.isalpha() for c in line) / len(line)
            if alpha_ratio > 0.3:
                filtered_lines.append(line)

    # Join lines and normalize whitespace
    text = '\n'.join(filtered_lines)
    text = re.sub(r'\s+', ' ', text)  # Normalize whitespace within lines
    text = re.sub(r'\n\s*\n+', '\n\n', text)  # Normalize paragraph breaks

    # Handle comic-specific dialogue markers
    text = re.sub(r'^\s*-\s*', '', text, flags=re.MULTILINE)  # Remove leading hyphens for dialogue

    text = text.strip()

    return text


In [ ]:
def extract_and_clean_comic_text(image_path):
    raw_text = extract_comic_text(image_path)
    cleaned_text = clean_comic_text(raw_text)
    return cleaned_text


In [ ]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
class ComicDataset(Dataset):
    def __init__(self, json_file, image_dir, tokenizer):
        with open(json_file) as f:
            self.data = json.load(f)

        self.max_chunks = 5
        self.image_dir = image_dir
        self.tokenizer = tokenizer
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(config.image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

        # Flatten chunks
        self.chunks = []
        for comic in self.data['comics']:
            for chunk in comic['chunks']:
                self.chunks.append({
                    'comic_id': comic['comic_id'],
                    'chunk_id': chunk['chunk_id'],
                    'summary': chunk['summary'],
                    'pages': chunk['pages'],
                    'page_range': chunk['page_range']
                })

    def __len__(self):
        return len(self.chunks)

    def __getitem__(self, idx):
        chunk = self.chunks[idx]

        # Load and process images
        images = []
        ocr_texts = []
        valid_pages = []

        for page in chunk['pages'][:config.max_images]:
            img_path = os.path.join(self.image_dir, chunk['comic_id'], page)
            try:

                if not os.path.exists(img_path):
                    raise FileNotFoundError(f"Image file not found: {img_path}")
                    continue

                with Image.open(img_path) as img:
                    img = img.convert('RGB')
                    images.append(self.transform(img))
                    ocr_text = extract_and_clean_comic_text(img_path)
                    ocr_texts.append(ocr_text)
                    valid_pages.append(page)

                images.append(self.transform(img))

                ocr_text = extract_and_clean_comic_text(img_path)
                ocr_texts.append(ocr_text)

            except Exception as e:
                print(f"[ERROR] Loading {img_path}: {e}")
                continue

        # Pad images
        while len(images) < config.max_images:
            images.append(torch.zeros(3, config.image_size, config.image_size))
            ocr_texts.append("")

        images = images[:config.max_images]
        ocr_texts = ocr_texts[:config.max_images]

        images = torch.stack(images)

        # Track number of real images
        num_real_images = len(images)

        # Process text
        text_input = f"Summarize pages {chunk['page_range']}:"
        tokenized = self.tokenizer(
            text_input,
            padding='max_length',
            truncation=True,
            max_length=config.max_len,
            return_tensors='pt'
        )

        # Tokenize each OCR chunk separately (no padding, no special tokens)
        tokenized_chunks = [
            self.tokenizer(
                text,
                add_special_tokens=False,
                truncation=True,
                max_length=config.max_len,  # Optional per chunk truncation
            )['input_ids']
            for text in ocr_texts
        ]

        # Track start and end indices for each chunk
        chunk_indices = []
        current = 1  # Start after BOS
        for chunk1 in tokenized_chunks:
            start = current
            end = current + len(chunk1)  # exclusive
            chunk_indices.append((start, end))
            current = end

        #if overflow
        chunk_indices = chunk_indices[:self.max_chunks]
        # Pad ocr_chunk_indices
        while len(chunk_indices) < self.max_chunks:
            chunk_indices.append((0, 0))  # Pad with dummy (0, 0) indices

        # Flatten the tokenized input and manually add BOS and EOS
        combined_input_ids = (
            [self.tokenizer.bos_token_id] +
            [token_id for chunk in tokenized_chunks for token_id in chunk] +
            [self.tokenizer.eos_token_id]
        )

        # Truncate final sequence if needed
        combined_input_ids = combined_input_ids[:config.max_len]

        # Pad if shorter
        pad_len = config.max_len - len(combined_input_ids)
        if pad_len > 0:
            combined_input_ids += [self.tokenizer.pad_token_id] * pad_len

        # If truncation cut off some chunks, adjust chunk_indices
        new_chunk_indices = []
        for start, end in chunk_indices:
            # If start is beyond the max_len, skip this chunk
            if start >= config.max_len - 1:
                # break
                new_chunk_indices.append((config.max_len, config.max_len))
                continue
            # Clamp end to max_len - 1 (to leave room for EOS)
            end = min(end, config.max_len - 1)
            new_chunk_indices.append((start, end))
        chunk_indices = new_chunk_indices
        chunk_indices = torch.tensor(chunk_indices)

        tokenized_ocr_tensor = torch.tensor(combined_input_ids)

        attention_mask = torch.ones_like(tokenized_ocr_tensor)

        # Process summary (target)
        summary = chunk['summary']
        if summary.startswith("none "):  # Remove "none " prefix if present
            summary = summary[5:]

        # Tokenize summary
        tokenized_summary = self.tokenizer(
            summary,
            padding='max_length',
            truncation=True,
            max_length=config.max_len,
            return_tensors='pt'
        )

        # Create decoder input for training (shifted right)
        decoder_input_ids = tokenized_summary['input_ids'].clone()
        decoder_input_ids = decoder_input_ids.squeeze(0)

        # Prepare shifted decoder input (teacher forcing)
        labels = decoder_input_ids.clone()
        decoder_input_ids = torch.cat([
            torch.tensor([self.tokenizer.bos_token_id]),
            decoder_input_ids[:-1]
        ])

        # tgt_key_padding_mask = (decoder_input_ids == self.tokenizer.pad_token_id).transpose(0, 1)
        tgt_key_padding_mask = (decoder_input_ids == self.tokenizer.pad_token_id)

        return {
            'ocr_input_ids': tokenized_ocr_tensor,
            'ocr_attention_mask': attention_mask,
            'images': images,
            'decoder_input_ids': decoder_input_ids,
            'decoder_attention_mask': tgt_key_padding_mask.unsqueeze(0),
            'ocr_chunk_indices': chunk_indices,
            'labels': labels,
            'comic_id': chunk['comic_id'],
            'chunk_id': chunk['chunk_id'],
            'num_real_images': num_real_images
        }

In [ ]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

dataset = ComicDataset(config.json_file_small, config.image_dir_small, tokenizer)
print(f"Dataset loaded with {len(dataset)} examples")

debug_loader = DataLoader(dataset, batch_size=1, shuffle=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

AttributeError: 'Config' object has no attribute 'json_file_small'

In [ ]:
def visualize_comic_sample(batch):

    images = batch['images'][0]  # [num_images, 3, H, W]

    ocr_input_text = tokenizer.batch_decode(batch['ocr_input_ids'], skip_special_tokens=False)

    summary = batch['raw_summary'][0]

    print(f"Comic ID: {batch['comic_id'][0]}")
    print(f"Chunk ID: {batch['chunk_id'][0]}")
    print(f"\nOCR Text (first 200 chars): {ocr_input_text[:200]}...")
    print(f"\nSummary: {summary}")

    fig, axes = plt.subplots(1, config.max_images, figsize=(20, 4))

    for i in range(config.max_images):
        img = images[i].permute(1, 2, 0).cpu().numpy()
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)

        if isinstance(axes, np.ndarray):
            axes[i].imshow(img)
            axes[i].set_title(f"Page {i+1}")
            axes[i].axis('off')
        else:
            axes.imshow(img)
            axes.set_title(f"Page {i+1}")
            axes.axis('off')

    plt.tight_layout()
    plt.show()

for i, batch in enumerate(debug_loader):
    visualize_comic_sample(batch)
    if i >= config.debug_samples - 1:
        break


NameError: name 'debug_loader' is not defined

In [ ]:
class LearnedPositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model):
        super().__init__()
        self.pos_embedding = nn.Embedding(max_len, d_model)

    def forward(self, x):
        seq_len = x.size(1)
        positions = torch.arange(0, seq_len, dtype=torch.long, device=x.device)
        positions = positions.unsqueeze(0).expand(x.size(0), seq_len)
        return x + self.pos_embedding(positions)

class CustomEncoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward):
        super().__init__()
        self.layer = TransformerEncoderLayer(d_model, nhead, dim_feedforward,  batch_first=True)

    def forward(self, src, src_mask=None, src_key_padding_mask=None ):
        return self.layer(src, src_mask=src_mask, src_key_padding_mask=src_key_padding_mask)

class CustomDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward):
        super().__init__()
        self.layer = TransformerDecoderLayer(d_model, nhead, dim_feedforward, batch_first=True)

    def forward(self, tgt, memory, tgt_mask=None, memory_mask=None, tgt_key_padding_mask=None, memory_key_padding_mask=None):
        return self.layer(tgt, memory, tgt_mask, memory_mask, tgt_key_padding_mask, memory_key_padding_mask)

In [ ]:
class CustomBART(nn.Module):
    def __init__(self, vocab_size, d_model=512, nhead=4, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=1024, max_len=768):
        super().__init__()
        self.nhead = nhead
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        self.position_encoding = LearnedPositionalEncoding(max_len, d_model)

        self.encoder_layers = nn.ModuleList([CustomEncoderLayer(d_model, nhead, dim_feedforward) for _ in range(num_encoder_layers)])
        self.decoder_layers = nn.ModuleList([CustomDecoderLayer(d_model, nhead, dim_feedforward) for _ in range(num_decoder_layers)])

        self.output_proj = nn.Linear(d_model, vocab_size)

    def forward(self, input_ids=None, decoder_input_ids=None, attention_mask=None,
            decoder_attention_mask=None, embeddings=None):

        if embeddings is not None:
            src_emb = embeddings
        else:
            src_emb = self.position_encoding(self.token_embedding(input_ids))

        if attention_mask is not None:
            attention_mask = attention_mask.bool()
        if decoder_attention_mask is not None:
            decoder_attention_mask = decoder_attention_mask.bool()
        # Encoder
        memory = src_emb
        for layer in self.encoder_layers:
            memory = layer(memory, src_key_padding_mask=attention_mask)

        # Decoder
        tgt_emb = self.position_encoding(self.token_embedding(decoder_input_ids))

        # Create causal mask for decoder
        tgt_len = decoder_input_ids.size(1)
        causal_mask = torch.triu(
            torch.ones(tgt_len, tgt_len, device=decoder_input_ids.device) * float('-inf'),
            diagonal=1
        )

        output = tgt_emb
        for layer in self.decoder_layers:
            output = layer(
                output, memory,
                tgt_mask=causal_mask,
                tgt_key_padding_mask=decoder_attention_mask,
                memory_key_padding_mask=attention_mask
            )

        return self.output_proj(output)

In [ ]:
class ResNetImageEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.feature_extractor = nn.Sequential(*list(resnet.children())[:-2])

    def forward(self, images):
        # images: [batch_size, num_images, 3, H, W]
        batch_size, num_images = images.shape[:2]
        images = images.view(batch_size * num_images, *images.shape[2:])
        features = self.feature_extractor(images)  # [batch*num_images, 2048, 7, 7]
        return features.view(batch_size, num_images, 2048, 7, 7)

In [ ]:
def interleave_image_text_embeddings(image_feats, text_embs, text_chunk_indices, num_real_images, max_len=768):
    B, _, D = text_embs.shape  # B=batch size, D=embedding dim
    chunk_count = text_chunk_indices.shape[1]
    final_embs = torch.zeros(B, max_len, D, device=text_embs.device)
    attention_mask = torch.zeros(B, max_len, dtype=torch.long, device=text_embs.device)

    for b in range(B):
        embed_ptr = 0  # track how far we’ve written

        for i in range(chunk_count):
            if i >= num_real_images[b]:
                break
            # image chunk (49 tokens)
            img_start = i * 49
            img_end = min(img_start + 49, image_feats.shape[1])
            img_chunk = image_feats[b, img_start:img_end]

            # Write image tokens
            img_len = img_chunk.size(0)
            if embed_ptr + img_len > max_len:
                break
            final_embs[b, embed_ptr:embed_ptr + img_len] = img_chunk
            attention_mask[b, embed_ptr:embed_ptr + img_len] = 1
            embed_ptr += img_len

            # Get corresponding text chunk
            t_start, t_end = text_chunk_indices[b, i]
                # Skip dummy chunks (they were added as (max_len, max_len))
            if t_start >= max_len:
                continue
            txt_chunk = text_embs[b, t_start:t_end]
            txt_len = txt_chunk.size(0)

            if embed_ptr + txt_len > max_len:
                available = max_len - embed_ptr
                txt_chunk = txt_chunk[:available]
                txt_len = txt_chunk.size(0)

            final_embs[b, embed_ptr:embed_ptr + txt_len] = txt_chunk
            attention_mask[b, embed_ptr:embed_ptr + txt_len] = 1
            embed_ptr += txt_len

    return final_embs, attention_mask


In [ ]:
class CustomBARTWithVision(nn.Module):
    def __init__(self, base_model: CustomBART, d_model=512):
        super().__init__()
        self.base_model = base_model

        # Vision encoder using ResNet's last convolutional layer (7x7 spatial features)
        self.image_encoder = ResNetImageEncoder()

        # Spatial feature projection
        self.image_proj = nn.Sequential(
            nn.Conv3d(2048, d_model, kernel_size=(1,1,1)),
            nn.ReLU(),
            nn.LayerNorm([d_model, config.max_images, 7, 7])
        )

        # Extended positional encoding for combined sequence
        self.extended_position_encoding = LearnedPositionalEncoding(
            config.max_len + config.max_images * 49,  # 7x7 spatial features per image
            d_model
        )

    def forward(self, input_ids, images, decoder_input_ids, attention_mask=None, decoder_attention_mask=None, ocr_chunk_indices=None, num_real_images=None):
        batch_size = images.shape[0]
        device = images.device


        image_features = self.image_encoder(images)  # [batch, num_images, 2048, 7, 7]

        image_features = self.image_proj(image_features.permute(0, 2, 1, 3, 4))  # [batch, d_model, num_images, 7, 7]

        # Reshape to sequence of tokens: [batch, num_images*49, d_model]
        image_features = image_features.view(batch_size, config.d_model, -1).permute(0, 2, 1)

        text_emb = self.base_model.token_embedding(input_ids)

        input_emb, input_attention = interleave_image_text_embeddings(image_features, text_emb, ocr_chunk_indices, num_real_images,max_len=config.max_len)
        input_emb = self.base_model.position_encoding(input_emb)

        return self.base_model(
            embeddings=input_emb,
            decoder_input_ids=decoder_input_ids,
            attention_mask=input_attention,
            decoder_attention_mask=decoder_attention_mask
        )

In [ ]:
def initialize_models():
    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

    base_model = CustomBART(
        vocab_size=tokenizer.vocab_size,
        d_model=config.d_model,
        nhead=config.nhead,
        num_encoder_layers=config.num_encoder_layers,
        num_decoder_layers=config.num_decoder_layers,
        dim_feedforward=config.dim_feedforward,
        max_len=config.max_text_len
    )

    checkpoint = torch.load(config.pretrained_bart_path, map_location='cpu')

    if "model_state_dict" in checkpoint:
        state_dict = checkpoint["model_state_dict"]
    else:
        state_dict = checkpoint

    new_state_dict = {}
    for key, value in state_dict.items():
        if key.startswith("module."):
            new_key = key[len("module."):]
        elif key.startswith("_orig_mod."):
            new_key = key[len("_orig_mod."):]
        else:
            new_key = key
        new_state_dict[new_key] = value

    base_model.load_state_dict(new_state_dict, strict=False)
    print(f"Loaded pretrained CustomBART weights from {config.pretrained_bart_path}")

    model = CustomBARTWithVision(base_model, d_model=config.d_model)

    model = model.to(device)

    return tokenizer, model


In [ ]:
def train_model(tokenizer=None, model=None, optimizer=None, loss_fn=None, scaler=None, dataset=None, resume=0, resume_batch_idx=0, seed = 42):

    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size

    generator = torch.Generator().manual_seed(seed)
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)

    train_sampler = torch.utils.data.RandomSampler(train_dataset, generator=generator)
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, sampler=train_sampler)

    # Training loop
    global_step = resume
    count = 0
    print(global_step)
    for epoch in range(config.num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

        for batch_idx, batch in enumerate(progress_bar):
            if epoch == 0 and batch_idx < resume_batch_idx:
              continue  # Skip already processed batches
            inputs = {
                'ocr_input_ids': batch['ocr_input_ids'].to(device),
                'ocr_attention_mask': batch['ocr_attention_mask'].to(device),
                'images': batch['images'].to(device)
            }
            labels = batch['labels'].to(device)

            # Mixed precision forward
            with autocast(device_type=device.type, dtype=torch.float16):
                decoder_attention_mask = batch['ocr_input_ids'].eq(tokenizer.pad_token_id).to(device)

                if 'attention_mask' in batch and batch['attention_mask'] is not None:
                    batch['attention_mask'] = batch['attention_mask'].bool()

                if 'decoder_attention_mask' in batch and batch['decoder_attention_mask'] is not None:
                    batch['decoder_attention_mask'] = batch['decoder_attention_mask'].bool()

                outputs = model(
                  input_ids=batch['ocr_input_ids'].to(device),
                  images=batch['images'].to(device),
                  decoder_input_ids=batch['decoder_input_ids'].to(device),
                  attention_mask=batch['ocr_attention_mask'].bool().to(device),
                  decoder_attention_mask=decoder_attention_mask,
                  ocr_chunk_indices=batch['ocr_chunk_indices'].to(device),
                  num_real_images=batch['num_real_images']
                )
                loss = loss_fn(outputs.view(-1, outputs.size(-1)), labels.view(-1))
                loss = loss/config.grad_accum_steps #for gradient accumulation
            # Visualization at regular intervals
            count += 1
            if count % 1 == 0:
                print(f"Global step {global_step}")

                # Get predictions
                with torch.no_grad():
                    predicted_ids = outputs.argmax(dim=-1)

                # Decode inputs, targets, and predictions
                ocr_input_text = tokenizer.batch_decode(batch['ocr_input_ids'], skip_special_tokens=False)
                decoder_input_text = tokenizer.batch_decode(batch['decoder_input_ids'], skip_special_tokens=False)
                predicted_text = tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

                # Handle labels for decoding
                labels_for_decode = batch['labels'].clone()
                labels_for_decode[labels_for_decode == -100] = tokenizer.pad_token_id
                target_text = tokenizer.batch_decode(labels_for_decode, skip_special_tokens=True)

                # Print visualization
                print("\n [Visualization]")
                print(f" Input         : {ocr_input_text[0]}")
                print(f" Target        : {target_text[0]}")
                print(f" Decoder Input : {decoder_input_text[0]}")
                print(f" Predicted     : {predicted_text[0]}")

            # Backward pass
            scaler.scale(loss).backward()

            # Optimize
            if (batch_idx + 1) % config.grad_accum_steps == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                global_step += 1

            total_loss += loss.item()
            progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

            if global_step % 10 == 0:
              if global_step == 0:
                continue
              torch.save({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'scaler_state_dict': scaler.state_dict(),
                        'step': global_step,
                        'batch_idx': batch_idx

                    }, f"{config.save_path}/checkpoint_weight1.pt")

        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scaler_state_dict': scaler.state_dict(),
            'train_loss': total_loss/len(train_loader),
            # 'val_loss': val_loss,
            'step': global_step
        }, f"{config.save_path}/checkpoint_weight1.pt")

    return model


In [ ]:
if __name__ == "__main__":

    tokenizer, model = initialize_models()
    dataset = ComicDataset(config.json_file, config.image_dir, tokenizer)

    optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
    loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
    scaler = torch.cuda.amp.GradScaler()

    checkpoint = torch.load("/content/drive/MyDrive/multimodal_summarization/checkpoint_weight.pt", map_location=device)

    global_step = 0
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)


    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])

    global_step = checkpoint['step'] + 1

    resume_batch_idx = checkpoint.get('batch_idx', 0) + 1
    print("resume_batch_idx = ", resume_batch_idx)
    model = train_model(tokenizer=tokenizer, model=model, optimizer=optimizer, loss_fn=loss_fn, scaler=scaler, dataset=dataset, resume=global_step, seed=44 )

    torch.save(model.state_dict(), os.path.join(config.save_path, "multimodal_comic_summarizer.pt"))

    tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
    test_dataset = ComicDataset(config.json_file, config.image_dir, tokenizer)
    test_loader = DataLoader(test_dataset, batch_size=config.batch_size)

    rouge_scores = evaluate_model(model, test_loader, tokenizer)
    print("\nFinal Evaluation Scores:")
    for key in rouge_scores:
        print(f"{key}:")
        print(f"  Precision: {rouge_scores[key]['precision']:.4f}")
        print(f"  Recall:    {rouge_scores[key]['recall']:.4f}")
        print(f"  F1:        {rouge_scores[key]['fmeasure']:.4f}")

Loaded pretrained CustomBART weights from /content/drive/MyDrive/multimodal_summarization/xsum_step_4500_1745223373.pt


<ipython-input-22-dd2da07f31a2>:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


resume_batch_idx =  879
631


Epoch 1:   0%|          | 0/91 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Global step 631

🔍 [Visualization]
📝 Input         : <s>Eae ee DOSE GOR the GME We NOT BEN bade mPEW-THOUBAND DOLLARS WA sneer YEAR.IT'S BEEN JEN SINCE Z WENT HUNTING, BUT... _ IZ; TRY NOT TO 7H 'I ABOUT It t HEATHER, LOT EASIER YOU" TAKE, IT ONE pay KNOW, IT'S pune Bay Brace fain Cte eran TO MAC wwe Fins' "ere OF COURSE, IF HE HADN'T CONG 'THAT HEP STILL STILL BE. IK THAT WAY, ea seen ALL UNFORTUNATELY THAT. WORK FOR ENERY- ONE, JUDD. T WAS RAISED ALWAYS TO PLAN AHEAD, ALWAYS TO HAVE EVERY TINGENCY ACCOUNTED DOESN'T HESE CIR Wwe SONG 1O ANYTHING TO ANGE, one ae OR THE IMSTANCES, y, How OLD ARE You/HEATHER? IENTY-EIGHT? TWENTY-NINE? AND YOU TALK Like You'Re LOCKED INTo PATTERNS THAT WOULD TAKE A LIFE TA TO BUILD <A IAUCH ONGER CIFE-TIME THAN YOUN T UNDeR- ens" a YOU'RE UGH) MAY THAN SH? BUT VER_MBAR A TRUER WORD THAN Ww DP atgane oF TIONS ARG STILL ENDLESS, z WAS pee la arta) EADY TO FALL ON HIS rene OVER THEM MESS HOUGHT HE'D GOT hin ELF INTO, BUT I PULLED HIM OUT OF HIS FUNK, AND WHE

...

Global step 632

🔍 [Visualization]
📝 Input         : <s>\...GOMETHING: VERY SERIOUS." IF THEY CATCH SCOTT AND JEAN, THEY'LL BE FORCED TO GO BACK. AND WE ALL KNOW WHAT'S WAITING FOR THEM. GREY. YOU THINK SENDING IT SUCKS. THEM BACK SUCKS. (OU MUST BE UGH TO STAND UP TO COMPLICATIONS. YOU'RE AN X-MAN. SIDING WITH THE X-MEN IS THE RATIONAL THING TO DO... S NV you MUST FIND THE COURAGE TO BE... IRRATIONAL.\...GOMETHING: VERY SERIOUS." IF THEY CATCH SCOTT AND JEAN, THEY'LL BE FORCED TO GO BACK. AND WE ALL KNOW WHAT'S WAITING FOR THEM. GREY. YOU THINK SENDING IT SUCKS. THEM BACK SUCKS. (OU MUST BE UGH TO STAND UP TO COMPLICATIONS. YOU'RE AN X-MAN. SIDING WITH THE X-MEN IS THE RATIONAL THING TO DO... S NV you MUST FIND THE COURAGE TO BE... IRRATIONAL.IRRATIONAL ENOUGH TO STAND UP FOR WHAT YOU BELIEVE IN. ENOUGH! ENOUGH WITH THE LISTENING. YOU ARE TAKING ME BACK RIGHT NOW. IRRATIONAL ENOUGH TO LOSE YOUR HEART TO A N [_WAIT, ARE YOU TRYING TO PUT SOME KIND OF WEIRD ALIEN SPELL ON ME? _f YOU MEA

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5962: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
Epoch 1:   8%|▊         | 7/91 [06:29<1:18:18, 55.94s/it, loss=1.1244]

Global step 632

🔍 [Visualization]
📝 Input         : <s>Sey DO YOU HAVE E&/ A MAP OF A MAP? 4 THE UNITED STATES? OR THE WORLD? (4 SOMETHING [i OR--WOW! YOU KNOW WHO YOU LOOK LIKE?Sey DO YOU HAVE E&/ A MAP OF A MAP? 4 THE UNITED STATES? OR THE WORLD? (4 SOMETHING [i OR--WOW! YOU KNOW WHO YOU LOOK LIKE?You Look Fert] JUST LIKE [2 a — FIVE DOLLARS?! FOR A MAGAZINE?!You Look Fert] JUST LIKE [2 a — FIVE DOLLARS?! FOR A MAGAZINE?!GET YOUR BUTT BACK ON MY BIKE AND BACK TO THE SCHOOL. IT'S NOT FAIR THAT EVERYONE IS BLAMING ME FOR SOMETHING T A DEAL WITH THE PEOPLE OF THIS KEEP OUR MUTANT CRAZY DOWN TOA MINIMUM. YOU'RE HERE LESS THAN 24 HOURS AND ALREADY YOU'RE ANNOYING ME MORE THAN is, T MUST 5 aA 'AH, UNBREAKABLE W BONES, SO WHY SO I CAN PUT YOU IN A HEADLOCK AND SPANK YOU OR YOU CAN JUST GET BACK ON THE BIKE AND-- SLEEP TI THINK, COOLER HEADS I= AND ALL THAT, THAT YOU GUYS NEED TO GO BACK WHERE YOU CAME FROM. T'M THE GUY THAT XAVIER PUT ' OF THE SCHOOL BEFORE YOU MURDERED HIM. CHARGE I DIDN'

Epoch 1:   8%|▊         | 7/91 [07:13<1:26:45, 61.97s/it, loss=1.1244]


KeyboardInterrupt: 

In [ ]:
def evaluate_model(model, test_loader, tokenizer):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            # Prepare inputs
            inputs = {
                'input_ids': batch['ocr_input_ids'].to(device),
                'images': batch['images'].to(device),
                'decoder_input_ids': batch['decoder_input_ids'].to(device),
                'attention_mask': batch['ocr_attention_mask'].bool().to(device),
                'decoder_attention_mask': batch['ocr_input_ids'].eq(tokenizer.pad_token_id).to(device),
                'ocr_chunk_indices': batch['ocr_chunk_indices'].to(device),
                'num_real_images': batch['num_real_images']
            }

            # Get labels (convert -100 to pad token id for decoding)
            labels = batch['labels'].clone()
            labels[labels == -100] = tokenizer.pad_token_id
            labels = labels.cpu().numpy()

            # Generate summaries using greedy decoding
            batch_size = inputs['input_ids'].size(0)
            generated = torch.full((batch_size, 1), tokenizer.bos_token_id, dtype=torch.long, device=device)

            for _ in range(config.max_text_len):
                outputs = model(
                    input_ids=inputs['input_ids'],
                    images=inputs['images'],
                    decoder_input_ids=generated,
                    attention_mask=inputs['attention_mask'],
                    decoder_attention_mask=None,  # Don't need mask for generation
                    ocr_chunk_indices=inputs['ocr_chunk_indices'],
                    num_real_images=inputs['num_real_images']
                )

                next_token_logits = outputs[:, -1, :]
                next_tokens = torch.argmax(next_token_logits, dim=-1).unsqueeze(1)
                generated = torch.cat([generated, next_tokens], dim=-1)

                # Stop if all sequences generated EOS
                if (next_tokens == tokenizer.eos_token_id).all():
                    break

            # Decode predictions and references
            preds = tokenizer.batch_decode(generated, skip_special_tokens=True)
            refs = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Store for scoring
            all_preds.extend(preds)
            all_labels.extend(refs)

    # Calculate ROUGE scores
    rouge_scores = {
        'rouge1': {'precision': 0, 'recall': 0, 'fmeasure': 0},
        'rouge2': {'precision': 0, 'recall': 0, 'fmeasure': 0},
        'rougeL': {'precision': 0, 'recall': 0, 'fmeasure': 0}
    }

    for pred, ref in zip(all_preds, all_labels):
        scores = scorer.score(ref, pred)
        for key in scores:
            rouge_scores[key]['precision'] += scores[key].precision
            rouge_scores[key]['recall'] += scores[key].recall
            rouge_scores[key]['fmeasure'] += scores[key].fmeasure

    # Average scores
    for key in rouge_scores:
        for metric in ['precision', 'recall', 'fmeasure']:
            rouge_scores[key][metric] /= len(all_preds)


    with open(f"{config.save_path}/evaluation_results.json", 'w') as f:
        json.dump(rouge_scores, f, indent=2)

    print(f"Results saved to {config.save_path}/evaluation_results.json")

    return rouge_scores

In [ ]:
tokenizer, model = initialize_models()

checkpoint_path = "/content/drive/MyDrive/multimodal_summarization/checkpoint_weight.pt"
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'], strict=False)
print(f"Loaded model from {checkpoint_path}")

test_dataset = ComicDataset(config.test_json_file, config.test_image_dir, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size)


Loaded pretrained CustomBART weights from /content/drive/MyDrive/multimodal_summarization/xsum_step_4500_1745223373.pt


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 206MB/s]


Loaded model from /content/drive/MyDrive/multimodal_summarization/checkpoint_weight.pt


In [ ]:
rouge_scores = evaluate_model(model, test_loader, tokenizer)
print(rouge_scores)

Evaluating:  86%|████████▋ | 139/161 [3:14:29<32:36, 88.94s/it]

[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/2.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/2.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/3.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/3.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/4.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/4.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/5.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/5.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/x-tr

Evaluating:  87%|████████▋ | 140/161 [3:15:21<27:18, 78.03s/it]

[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/16.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/16.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/17.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/17.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/18.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/18.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/19.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/x-treme-x-men-v1-010-2002/19.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_ima

Evaluating:  88%|████████▊ | 141/161 [3:15:56<21:42, 65.12s/it]

...

[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/1.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/1.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/2.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/2.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/3.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/3.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/4.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/4.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_ima

Evaluating:  99%|█████████▉| 160/161 [3:23:58<00:25, 25.41s/it]

[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/21.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/21.jpg'
[ERROR] Loading /content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/22.jpg: cannot identify image file '/content/drive/MyDrive/multimodal_summarization/test_images/young-avengers-v1-011-2006/22.jpg'


Evaluating: 100%|██████████| 161/161 [3:24:14<00:00, 76.11s/it]


Results saved to /content/drive/MyDrive/multimodal_summarization//evaluation_results.json
{'rouge1': {'precision': 0.0282035771226767, 'recall': 0.07814421995810315, 'fmeasure': 0.03731312860417132}, 'rouge2': {'precision': 0.003080877952099227, 'recall': 0.0083484285707677, 'fmeasure': 0.004070214584896119}, 'rougeL': {'precision': 0.0282035771226767, 'recall': 0.07814421995810315, 'fmeasure': 0.03731312860417132}}


In [ ]:
def verify_image_dataset(image_dir):

    from tqdm import tqdm
    corrupt_files = []

    for root, _, files in os.walk(image_dir):
        for file in tqdm(files, desc="Checking images"):
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                path = os.path.join(root, file)
                try:
                    img = Image.open(path)
                    img.verify()  # Verify without loading
                except Exception as e:
                    corrupt_files.append((path, str(e)))

    if corrupt_files:
        print(f"\nFound {len(corrupt_files)} corrupt images:")
        for path, error in corrupt_files[:10]:  # Print first 10
            print(f"{path}: {error}")
        return False
    return True

# Run verification (optional)
verify_image_dataset(config.test_image_dir)

Checking images: 0it [00:00, ?it/s]
Checking images:  13%|█▎        | 3/23 [00:04<00:27,  1.38s/it]


KeyboardInterrupt: 

In [ ]:
def generate_summaries(model, tokenizer, dataset, num_examples=5):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    examples = []

    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            if i >= num_examples:
                break

            # Prepare inputs
            inputs = {
                'input_ids': batch['ocr_input_ids'].to(device),
                'images': batch['images'].to(device),
                'attention_mask': batch['ocr_attention_mask'].bool().to(device),
                'ocr_chunk_indices': batch['ocr_chunk_indices'].to(device),
                'num_real_images': batch['num_real_images']
            }

            # Get reference summary (convert -100 to pad token id)
            labels = batch['labels'].clone()
            labels[labels == -100] = tokenizer.pad_token_id

            # Generate summary using greedy decoding
            generated = torch.full((1, 1), tokenizer.bos_token_id, dtype=torch.long, device=device)

            for _ in range(config.max_text_len):
                outputs = model(
                    input_ids=inputs['input_ids'],
                    images=inputs['images'],
                    decoder_input_ids=generated,
                    attention_mask=inputs['attention_mask'],
                    decoder_attention_mask=None,  # Don't need mask for generation
                    ocr_chunk_indices=inputs['ocr_chunk_indices'],
                    num_real_images=inputs['num_real_images']
                )

                next_token = outputs[:, -1, :].argmax(dim=-1).unsqueeze(1)
                generated = torch.cat([generated, next_token], dim=-1)

                # Stop if EOS generated
                if next_token == tokenizer.eos_token_id:
                    break

            # Decode results
            predicted_summary = tokenizer.decode(generated[0], skip_special_tokens=True)
            reference_summary = tokenizer.decode(labels[0], skip_special_tokens=True)

            examples.append({
                'comic_id': batch['comic_id'][0],
                'chunk_id': batch['chunk_id'][0],
                'predicted_summary': predicted_summary,
                'reference_summary': reference_summary
            })

            print(f"\nExample {i+1}:")
            print(f"Comic ID: {batch['comic_id'][0]}, Chunk ID: {batch['chunk_id'][0]}")
            print(f"Predicted: {predicted_summary}")
            print(f"Reference: {reference_summary}")

    return examples

In [ ]:
neotest_dataset = ComicDataset(config.json_file, config.image_dir, tokenizer)
neotest_loader = DataLoader(neotest_dataset, batch_size=config.batch_size)
examples = generate_summaries(model, tokenizer, neotest_dataset, num_examples=5)


Example 1:
Comic ID: 2099-unlimited-001, Chunk ID: 8
Predicted: 
Reference: Sweet Dreams’ armored killers arrive at Quirk’s home, kill her boyfriend and point their guns on her. Hulk breaks through they all, takes the bullets meant for her and clobbers the killers.

Example 2:
Comic ID: all-new-doop-002-2014, Chunk ID: 4
Predicted: The X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the middle of the night. The X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the middle of the night, and the X-Men are in the midd

In [ ]:
examples = generate_summaries(model, tokenizer, test_dataset, num_examples=5)


Example 1 (beam_search):
Comic ID: avengers-millennium-tpb-part-1, Chunk ID: 33
Predicted: He asks if he is going to be able to help him. He tells her that she doesn’t know what it was like, but she has no idea how long they are in each other. She says that this isn’t a good time for them to get out of the way, and that there is something else to do about it. They have to go back to work together and find out who they are now. But they can't let themselves go back to work again because they don’t want to make sure they aren’t doing anything wrong with them anymore. And then we need to get out of here and see what happens when they're all right. It’s only one thing to worry about, so maybe they should take care of their new boss.
Reference: 

Example 2 (beam_search):
Comic ID: avengers-west-coast-089, Chunk ID: 2
Predicted: 
Reference: Julia “Spider-Woman” Carpenter asks what other choice there is, and turns to USAgent, declaring that although she hasn’t been an Avenger as long as he h